In [1]:
import glob
import os
import numpy as np
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

%matplotlib inline

import plotly
plotly.offline.init_notebook_mode(connected=True)

import sys
sys.path.append("..")

import mysql.connector as mariadb
from astropy.time import Time

from thor import clusterAndLink
from thor import analyzeClusters
from thor import analyzeProjections
from thor import analyzeObservations
from thor import calcLinkageFindable
from thor import calcLinkageEfficiency
from thor import propagateTestParticle
from thor import runRangeAndShiftOnVisit

from thor.plotting import plotProjectionVelocitiesFindable
from thor.plotting import plotProjectionVelocitiesFound
from thor.plotting import plotProjectionVelocitiesMissed
from thor.plotting import plotOrbitsFindable
from thor.plotting import plotOrbitsFound
from thor.plotting import plotOrbitsMissed
from thor.plotting import plotProjections
from thor.plotting import plotObservations

from thor import Config

In [2]:
# Connect to database
con = mariadb.connect(user='ztf', database='ztf')

In [ ]:
# https://zwickytransientfacility.github.io/ztf-avro-alert/schema.html

In [3]:
# Read alerts for solar system objects from after the photometry fix 
sso_alert_fix_date1 = Time('2018-05-16T23:30:00', format='isot', scale='utc') # first attribution fix
sso_alert_fix_date2 = Time('2018-06-08T23:30:00', format='isot', scale='utc') # second attribution fix
sso_alert_phot_fix_date = Time('2018-06-18T23:30:00', format='isot', scale='utc') # photometry fix date

jd_good = sso_alert_phot_fix_date.jd
#ssdistnr >= 0 
df = pd.read_sql_query('select * from alerts where jd > {} and ssdistnr >= 0'.format(jd_good), con)
print(len(df))

628667


In [5]:
df["ssnamenr"].nunique()

90104

In [7]:
df["field"].value_counts()

437     17613
448     15566
436     15450
397     14211
612     13818
503     12838
498     11613
449     11156
502     11138
563     10921
500     10520
450     10193
398      9634
497      9526
504      9260
501      8125
659      8071
554      7856
395      7334
496      7324
499      7303
447      7243
611      7210
555      6978
451      6678
553      6575
399      6291
445      6026
345      5893
692      5739
        ...  
1688        3
757         3
753         3
291         3
461         3
751         2
834         2
852         2
787         2
1540        2
1808        2
756         2
675         2
857         2
573         2
1693        1
841         1
851         1
822         1
716         1
866         1
817         1
767         1
850         1
1806        1
818         1
867         1
1744        1
1641        1
783         1
Name: field, Length: 499, dtype: int64

In [ ]:
columnMapping = {        
        
        # Observation ID
        "obs_id" : "obs_id",
        
        # Exposure time
        "exp_mjd" : "exp_mjd",
        
        # Visit ID
        "visit_id" : "visit_id",
        
        # Field ID
        "field_id" : "field",
        
        # Field RA in degrees
        "field_RA_deg" : "fieldRA_deg",
        
        # Field Dec in degrees
        "field_Dec_deg" : "fieldDec_deg",
        
        # Night number
        "night": "nid",
        
        # RA in degrees
        "RA_deg" : "ra",
        
        # Dec in degrees
        "Dec_deg" : "decl",
        
        # Observer's x coordinate in AU
        "obs_x_au" : "HEclObsy_X_au",
        
        # Observer's y coordinate in AU
        "obs_y_au" : "HEclObsy_Y_au",
        
        # Observer's z coordinate in AU
        "obs_z_au" : "HEclObsy_Z_au",
        
        # Magnitude (UNUSED)
        "mag" : "magpsf",
        
        ### Truth Parameters
        
        # Object name
        "name" : "ssnamenr",
        
        # Observer-object distance in AU
        "Delta_au" : "Delta_au",
        
        # Sun-object distance in AU (heliocentric distance)
        "r_au" : "r_au",
        
        # Object's x coordinate in AU
        "obj_x_au" : "HEclObj_X_au",
        
        # Object's y coordinate in AU
        "obj_y_au" : "HEclObj_Y_au",
        
        # Object's z coordinate in AU
        "obj_z_au" : "HEclObj_Z_au",
        
        # Object's x velocity in AU per day
        "obj_dx/dt_au_p_day" : "HEclObj_dX/dt_au_p_day",
        
        # Object's y velocity in AU per day
        "obj_dy/dt_au_p_day" : "HEclObj_dY/dt_au_p_day",
        
        # Object's z velocity in AU per day
        "obj_dz/dt_au_p_day" : "HEclObj_dZ/dt_au_p_day",
        
        # Semi-major axis
        "a_au" : "a_au",
        
        # Inclination
        "i_deg" : "i_deg",
        
        # Eccentricity
        "e" : "e",
    }

def prepareZTFAlertsForTHOR(alerts):
    # Sort by field and exposure time
    alerts.sort_values(by=["field", "jd"], inplace=True)
    
    # Add exposure time in MJD column
    times = Time(alerts["jd"], format="jd", scale="utc")
    times = times.mjd
    alerts["exp_mjd"] = times
    
    # Only keep columns we want
    alerts = alerts[["ra", "decl", "field", "nid", "jd", "exp_mjd", "magpsf", "sigmapsf", "fid", "ssnamenr"]]
    
    # Read footprint and get field center locations
    footprint = pd.read_csv("/epyc/projects/rascals/ztf_information/field_grid/ZTF_Fields.txt",
                        delim_whitespace=True, 
                        skiprows=1,
                        names=["field", "fieldRA_deg", "fieldDec_deg", "Ebv", "Gal Long", "Gal Lat", "Ecl Long", "Ecl Lat" ,"Entry"])
    footprint = footprint[["field", "fieldRA_deg", "fieldDec_deg"]]
    
    os.environ["OORB_DATA"] = "/astro/users/moeyensj/.conda/envs/python3-moeyensj-thor/share/openorb"
    print(os.environ["OORB_DATA"])

    survey = pd.DataFrame({"exp_mjd": alerts["exp_mjd"].unique(), "visit_id" : np.arange(1, len(alerts["exp_mjd"].unique()) + 1)})
    eph = propagateTestParticle([1, 1, 0, 0.0002, 0.0002, 0.000], 59580, survey["exp_mjd"].values, observatoryCode="I41")
    survey = survey.merge(eph[["mjd", 'HEclObsy_X_au', 'HEclObsy_Y_au', 'HEclObsy_Z_au']], left_on="exp_mjd", right_on="mjd")
    
    observations = alerts.merge(footprint, left_on="field", right_on="field")
    observations = observations.merge(survey, on="exp_mjd")

    observations["obs_id"] = np.arange(1, len(observations) + 1)
    
    return observations

In [ ]:
os.environ["OORB_DATA"] = "/astro/users/moeyensj/.conda/envs/python3-moeyensj-thor/share/openorb"
print(os.environ["OORB_DATA"])

In [ ]:
eph = propagateTestParticle([1, 1, 0, 0.002, 0.002, 0.000], 59580, [59581])

In [ ]:
eph

In [ ]:
observations = prepareZTFAlertsForTHOR(df)

In [ ]:
observations.to_csv("../analysis/ztf/observations.txt", sep=" ", index=False)

In [ ]:
observations = pd.read_csv("../analysis/ztf/observations.txt", index_col=False, sep=" ", low_memory=False)

In [ ]:
observations[["field", "visit_id"]]["field"].value_counts()

In [ ]:
observations[observations["field"] == 766]["visit_id"].unique()

In [ ]:
plotObservations(observations[observations["field"] == 766], usePlotly=False, columnMapping=columnMapping)

In [ ]:
3330, 3331, 3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340,
       3341, 3342, 3343, 3344, 3345, 3346, 3347, 3348, 3349, 3350, 3351,
       3352, 3353, 3354, 3355, 3356, 3357, 3358, 3359, 3360, 3361, 3362,
       3363, 3364, 3365, 3366, 3367, 3368, 3369, 3370, 3371, 3372, 3373,
       3374, 3375, 3376, 3377, 3378, 3379, 3380, 3381, 3382, 3383, 3384,
       3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395,
       3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406,
       3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417,
       3418, 3419, 3420, 3421, 3422

In [ ]:
observations[~observations[columnMapping["name"]].isna()][columnMapping["name"]].nunique()

In [ ]:
projected_obs = runRangeAndShiftOnVisit(observations, 3330, 2.8, [-0.006, -0.006, 0], useAverageObject=False, cellArea=100, cellShape="circle", columnMapping=columnMapping, verbose=False)

In [ ]:
allObjects_projection, summary_projection = analyzeProjections(projected_obs, columnMapping=columnMapping)

In [ ]:
vxRange = [-0.1, 0.1]
vyRange = [-0.1, 0.1]

In [ ]:
fig, ax = plotProjectionVelocitiesFindable(allObjects_projection, vxRange=vxRange, vyRange=vyRange)
ax.set_xlim(-0.2, 0.2)
ax.set_ylim(-0.2, 0.2)

In [ ]:
allClusters_projection, clusterMembers_projection = clusterAndLink(
        projected_obs, 
        eps=0.005,
        minSamples=5, 
        vxRange=vxRange,
        vyRange=vyRange,
        vxBins=300,
        vyBins=300,
        threads=10,
        columnMapping=columnMapping)

In [ ]:
allClusters_projection, clusterMembers_projection, allObjects_projection, summary_projection = analyzeClusters(
            projected_obs,
            allClusters_projection, 
            clusterMembers_projection,
            allObjects_projection, 
            summary_projection,
            minSamples=5, 
            contaminationThreshold=0.0,
            columnMapping=columnMapping)

In [ ]:
observations["nid"]